In [1]:
import os
import cv2
import numpy as np
from tqdm import tqdm
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from skimage import feature
import pickle
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split

In [2]:
def extract_label(filename):
    if "frban" in filename:
        return "fresh banana"
    elif "frgrapp" in filename:
        return "fresh green apple"
    elif "rotapp" in filename:
        return "rotten apple"
    elif "rotban" in filename:
        return "rotten banana"
    elif "frredapp" in filename:
        return "fresh red apple"
    elif "frbg" in filename:
        return "fresh bitter gourd"
    elif "frcapsicum" in filename:
        return "fresh capsicum"
    elif "rotorange" in filename:
        return "rotten orange"
    elif "rotcapsicum" in filename:
        return "rotten capsicum"
    elif "rotbg" in filename:
        return "rotten bitter gourd"
    elif "frtomato" in filename:
        return "fresh tomato"
    elif "frorange" in filename:
        return "fresh orange"
    else:
        print(f"Unknown label for image: {filename}")
        return None

In [3]:
def extract_color_histogram(image, bins=(8, 8, 8)):
    hist = cv2.calcHist([image], [0, 1, 2], None, bins, [0, 256, 0, 256, 0, 256])
    hist = cv2.normalize(hist, hist).flatten()
    return hist

def extract_edges(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 100, 200)
    return edges.flatten()

def compute_texture_gradients(image):
    gray_image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    
    # Compute texture gradients using Sobel operator
    sobel_x = cv2.Sobel(gray_image, cv2.CV_64F, 1, 0, ksize=3)
    sobel_y = cv2.Sobel(gray_image, cv2.CV_64F, 0, 1, ksize=3)
    
    # Compute magnitude of gradients
    gradient_magnitude = np.sqrt(sobel_x**2 + sobel_y**2)
    
    # Compute mean and standard deviation of gradient magnitude
    mean_gradient = np.mean(gradient_magnitude)
    std_gradient = np.std(gradient_magnitude)
    
    # Compute texture gradients feature vector
    texture_gradients = np.array([mean_gradient, std_gradient])
    
    return texture_gradients

def extract_features(image):
    color_hist = extract_color_histogram(image)
    ta = compute_texture_gradients(image)
    edges = extract_edges(image)
    feature_vector = np.concatenate([color_hist, ta, edges])
    return feature_vector

In [4]:
def load_X_train_and_y_train_from_folder(folder_path):
    X_train = []
    y_train = []
    filenames = os.listdir(folder_path)
    for filename in tqdm(filenames, desc="Loading data"):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            image_path = os.path.join(folder_path, filename)
            image = cv2.imread(image_path)
            if image is not None:
                feature_vector = extract_features(image)
                y_train.append(extract_label(filename))
                X_train.append(feature_vector)
            else:
                print(f"Failed to read image: {image_path}")
    return X_train, y_train

In [5]:
def evaluate_model(true_y_train, numeric_predicted_y_train):    
    accuracy = accuracy_score(true_y_train, numeric_predicted_y_train)
    f1 = f1_score(true_y_train, numeric_predicted_y_train, average='micro')
    precision = precision_score(true_y_train, numeric_predicted_y_train, average='None')
    recall = recall_score(true_y_train, numeric_predicted_y_train, average='weighted')
    conf_matrix = confusion_matrix(true_y_train, numeric_predicted_y_train)
    
    return accuracy, f1, precision, recall, conf_matrix

In [6]:
def resize_X_train_in_folder(input_folder, output_folder, target_size=(224, 224)):
    filenames = os.listdir(input_folder)
    for filename in tqdm(filenames, desc="Resizing X_train"):
        if filename.endswith(".jpg") or filename.endswith(".png"):  
            input_image_path = os.path.join(input_folder, filename)
            output_image_path = os.path.join(output_folder, filename)
            image = cv2.imread(input_image_path)
            if image is not None:
                resized_image = cv2.resize(image, target_size)  
                cv2.imwrite(output_image_path, resized_image)
            else:
                pass

In [7]:
folder_path = "test_resized"
X_test, y_test = load_X_train_and_y_train_from_folder(folder_path)

Loading data:   0%|          | 0/821 [00:00<?, ?it/s]

Loading data: 100%|██████████| 821/821 [00:02<00:00, 286.95it/s]


In [8]:
with open('new3.pkl', 'rb') as file:
    xgb = pickle.load(file)

In [9]:
label_to_number = {"fresh banana": 0, "fresh green apple": 1, "rotten apple": 2, "rotten banana": 3, "fresh red apple": 4, "fresh bitter gourd": 5, "fresh capsicum": 6, "fresh orange": 7, "fresh tomato": 8, "rotten bitter gourd": 9, "rotten capsicum": 10, "rotten orange": 11}
numeric_y_test = [label_to_number[label] for label in y_test]

In [10]:
y_pred = xgb.predict(X_test)

predicted_y_test = [list(label_to_number.keys())[label] for label in y_pred]

In [11]:
correct_predictions = sum(1 for true_label, predicted_label in zip(y_test, predicted_y_test) if true_label == predicted_label)
total_test_samples = len(y_test)
accuracy = correct_predictions / total_test_samples
print("Accuracy:", accuracy)


Accuracy: 0.8599269183922046


In [12]:
numeric_y_test_arr = np.array(numeric_y_test)

In [13]:
accuracy_score(numeric_y_test_arr, y_pred)

0.8599269183922046

In [14]:
f1_score(numeric_y_test_arr, y_pred, average=None)

array([0.96618357, 0.99507389, 0.72164948, 0.96      , 0.81443299,
       0.25714286, 0.4516129 , 0.98507463, 0.82352941, 0.        ,
       0.97142857, 0.96153846])

In [15]:
f1_score(numeric_y_test_arr, y_pred, average='micro')

0.8599269183922046

In [16]:
precision_score(numeric_y_test_arr, y_pred, average=None)

array([0.93457944, 0.99019608, 0.76086957, 0.96969697, 0.85869565,
       0.14754098, 1.        , 0.99      , 0.7       , 0.        ,
       1.        , 0.92592593])

In [17]:
recall_score(numeric_y_test_arr, y_pred, average=None)

array([1.        , 1.        , 0.68627451, 0.95049505, 0.7745098 ,
       1.        , 0.29166667, 0.98019802, 1.        , 0.        ,
       0.94444444, 1.        ])

In [18]:
confusion_matrix(numeric_y_test_arr, y_pred)

array([[100,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0, 101,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  7,   1,  70,   3,  13,   0,   0,   0,   2,   0,   0,   6],
       [  0,   0,   2,  96,   0,   0,   0,   0,   0,   2,   0,   1],
       [  0,   0,  18,   0,  79,   0,   0,   1,   4,   0,   0,   0],
       [  0,   0,   0,   0,   0,   9,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,  51,  21,   0,   0,   0,   0,   0],
       [  0,   0,   1,   0,   0,   0,   0,  99,   0,   0,   0,   1],
       [  0,   0,   0,   0,   0,   0,   0,   0,  14,   0,   0,   0],
       [  0,   0,   1,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   1,   0,   0,   0,   0,  17,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0, 100]],
      dtype=int64)